Army Poker Sim
==============

In the army in 2023, we played a game that we (incorrectly) call "_Chinese Poker_".

During those games, several questions arose. This project aims to anser those questions, by simulating larget numbers of random games.

Questions
---------

1. How much does luck play a factor? (vs ideal strategy)
2. Is it better to focus on grouping, or flush/straight?
3. Is a _flush_ unfairly weighted?



In [1]:
# Setup

import pkgutil
from src import headToHead
from src import strategies

# Import strategies for comparison
package = strategies
prefix = package.__name__ + "."
allStrategies = []
for importer, modname, ispkg in pkgutil.iter_modules(package.__path__, prefix):
    allStrategies.append(__import__(modname, fromlist="dummy"))

ITERATIONS = 10000

We will put all of our strategies in a _round-robin_ tournament. Each match-up consists of 10000 games.


A win counts as 1 point (regardless of how many columns won).

In [2]:
import itertools
pairings = itertools.combinations(allStrategies, 2)

resultsTable = []

winners = []

def getPrettyName(nm):
    return nm.split(".")[-1]

for pairing in pairings:
    s1 = pairing[0]
    s2 = pairing[1]
    score = headToHead.head2Head(s1, s2, ITERATIONS)
    winner = s2
    loser = s1
    if score > 0:
        winner = s1
        loser = s2
        
    #print("%s beat %s" % (getPrettyName(winner.__name__), getPrettyName(loser.__name__)))
    winners.append(winner)

    winPercentage = int((((ITERATIONS - abs(score)) / 2) + abs(score) ) / ITERATIONS * 100)
    resultsTable.append([getPrettyName(winner.__name__), getPrettyName(loser.__name__), abs(winPercentage)])

results = {}
for strat in set(allStrategies):
    results[getPrettyName(strat.__name__)] = winners.count(strat)

Round-Robin Results
-------------------

In [3]:
import pandas as pd
pd.DataFrame(resultsTable, columns=["Winner", "Loser", "%"]).sort_values('%', ascending=False)


,Winner,Loser,%
2,preferGroupFlushStraight,chooseLeft,93
3,preferGroupStraightFlush,chooseLeft,93
4,preferGroups,chooseLeft,93
5,preferGroupsFlush,chooseLeft,93
0,preferFlushGroups,chooseLeft,90
6,preferStraightFlushGroup,chooseLeft,88
1,preferFlushStraightGroup,chooseLeft,86
21,preferGroupFlushStraight,preferStraightFlushGroup,63
27,preferGroupsFlush,preferStraightFlushGroup,63
14,preferGroupStraightFlush,preferFlushStraightGroup,62


In [4]:
def most_common(lst):
    return max(set(lst), key=lst.count)

print("The winner is %s" % (getPrettyName(most_common(winners).__name__)))

The winner is preferGroupsFlush
